In [9]:
import unicodedata
import io

file_path = "/home/noernova/Labs/tesstrain/data/shn-ground-truth/1729612954.13547.gt.txt"
with io.open(file_path, 'r', encoding='utf-8') as f:
    try:
        text = f.read()
        print(text)
    except UnicodeDecodeError as e:
        print(e)

ဢၼ်တေၸွၺ်ႈႁႂ်ႈသူဝူၼ်ႉၶႆႈလႆႈဝႃႈ


In [13]:
normalized_text = unicodedata.normalize('NFKD', text)
print(normalized_text)

ဢၼ်တေၸွၺ်ႈႁႂ်ႈသူဝူၼ်ႉၶႆႈလႆႈဝႃႈ
